In [5]:
#使用神经网络
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
## 基础工具
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time
import missingno as msno
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [6]:
#载入训练集和测试集；
path = './2sc/'
train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
test_data = pd.read_csv(path+'used_car_testB_20200421.csv', sep=' ')

In [7]:
train_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
train_data["notRepairedDamage"] = train_data["notRepairedDamage"].astype("float64")

In [8]:
test_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
test_data["notRepairedDamage"] = test_data["notRepairedDamage"].astype("float64")

In [9]:
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

In [10]:
train_data["power"].describe()

count    150000.000000
mean        119.316547
std         177.168419
min           0.000000
25%          75.000000
50%         110.000000
75%         150.000000
max       19312.000000
Name: power, dtype: float64

In [11]:
test_data["power"][test_data["power"]>600] = 600

In [12]:
#特征归一化
train_data["power"][train_data["power"]>600] = 600
features = train_data.columns.tolist()
features.remove("price")
features.remove("SaleID")
features

['name',
 'regDate',
 'model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'power',
 'kilometer',
 'notRepairedDamage',
 'regionCode',
 'seller',
 'offerType',
 'creatDate',
 'v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11',
 'v_12',
 'v_13',
 'v_14']

In [13]:
min_max_scaler = MinMaxScaler()
x = min_max_scaler.fit_transform(train_data[features].values)
x_test = min_max_scaler.transform(test_data[features].values)

In [14]:
y = train_data["price"].values

In [15]:
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [24]:
#搭建模型
model = keras.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")

In [17]:
#模型评估
print("训练集评估 MAE： ", mean_absolute_error(train_y, model.predict(train_x)))
print("测试集评估 MAE： ", mean_absolute_error(test_y, model.predict(test_x)))

训练集评估 MAE：  5919.243112109957
测试集评估 MAE：  5939.009354855792


In [18]:
#用全量数据训练
model = keras.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")
model.fit(x, y, batch_size=1024, epochs=200)

Epoch 1/100
147/147 [==============================] - 1s 10ms/step - loss: 5040.8789
Epoch 2/100
147/147 [==============================] - 1s 9ms/step - loss: 3267.5364
Epoch 3/100
147/147 [==============================] - 1s 9ms/step - loss: 1538.2218
Epoch 4/100
147/147 [==============================] - 1s 10ms/step - loss: 1177.5701
Epoch 5/100
147/147 [==============================] - 1s 10ms/step - loss: 1072.1326
Epoch 6/100
147/147 [==============================] - 1s 9ms/step - loss: 1007.7998
Epoch 7/100
147/147 [==============================] - 2s 11ms/step - loss: 957.6326
Epoch 8/100
147/147 [==============================] - 2s 11ms/step - loss: 918.5248
Epoch 9/100
147/147 [==============================] - 2s 11ms/step - loss: 885.3823
Epoch 10/100
147/147 [==============================] - 2s 11ms/step - loss: 858.6318
Epoch 11/100
147/147 [==============================] - 2s 10ms/step - loss: 835.2467
Epoch 12/100
147/147 [==============================] - 2s 1

In [19]:
#输出结果
predict_y = model.predict(x_test)

result = pd.DataFrame()
result["SaleID"] = test_data["SaleID"]
result["price"] = predict_y
#result["price"][result["price"] < 11] = 11
result.loc[result["price"] <11, "price"] = 11
result.loc[result["price"] >99999, "price"] = 99999
result.to_csv("ans_nn.csv", index=False)

In [21]:
#定义统计函数，方便了解数据分布
def show_stats(data):
    print("min： ", np.min(data))
    print("max： ", np.max(data))
    print("ptp： ", np.ptp(data))
    print("mean： ", np.mean(data))
    print("std： ", np.std(data))
    print("var： ", np.var(data))

In [22]:
show_stats(predict_y)

min：  -36.3267
max：  100750.7
ptp：  100787.03
mean：  5892.5933
std：  7390.361
var：  54617436.0


In [23]:
show_stats(result)

min：  SaleID    200000.0
price         11.0
dtype: float64
max：  SaleID    249999.0
price      99999.0
dtype: float64
ptp：  249988.0
mean：  SaleID    224999.500000
price       5892.579179
dtype: float64
std：  SaleID    14433.756727
price      7390.167904
dtype: float64
var：  SaleID    2.083333e+08
price     5.461458e+07
dtype: float64
